# Detailed comparison to Granger-causality and CCM

In [ ]:
import numpy as np
from synthetic_data import sample_points, beta, skfda_basis, spline_multi_sample
from causal import ccm_bivariate, granger, eval_candidate_DAGs
from kernels import K_ID
import pickle
import os
import matplotlib.pyplot as plt
from tqdm.notebook import tqdm

#### Data
Case 1:

In [ ]:
# beta function to move from linearity to non-linearity in relationship between X and Y

def beta_linear(s, t, a):
    c_1 = np.random.uniform(0.25, 0.75)
    c_2 = np.random.uniform(0.25, 0.75)
    return (10 - a) + a * (8 * (s - c_1) ** 2 - 8 * (t - c_2) ** 2)

In [ ]:
# historically dependent data
def hist_data_linear(X, upper_limit, a, pred_points, s_0):
    """
    Function to generate historically dependent data

    Inputs:
    X: (n_vars, n_samples * n_tests, n_obs) array of samples
    upper_limit: upper limit for predictions
    a: strength of dependence
    pred_points: prediction points
    linear: extend of linear dependence

    Returns:
    Y: (n_samples * n_tests, n_obs) response variable that is historically dependent on X
    """
    if len(X.shape)==2:
        X_arr = X.reshape(1, X.shape[0], X.shape[1])
    else:
        X_arr = X
    Y = np.zeros((X_arr.shape[1], len(pred_points[pred_points <= upper_limit])))
    s, t = np.meshgrid(pred_points[pred_points <= upper_limit], pred_points[pred_points <= upper_limit])
    for i in range(X_arr.shape[1]):  # looping over samples
        sum_y = np.zeros(len(pred_points[pred_points <= upper_limit]))
        for p in range(X_arr.shape[0]):  # looping over parent variables
            beta_p = beta_linear(s, t, a)
            y = np.zeros(len(pred_points[pred_points <= upper_limit]))
            for i_t, t in enumerate(pred_points[pred_points <= upper_limit]):  # looping over time points of y
                #y[i_t] = trapezoid(X_arr[p, i][:i_t+1] * beta_p[:i_t+1, i_t])
                if i_t >= s_0:
                    y[i_t] = np.sum(X_arr[p, i][(i_t-s_0):i_t] * beta_p[(i_t-s_0):i_t, i_t]) / i_t
                else:
                    y[i_t] = 0

            sum_y += y
            Y[i] = sum_y

    return Y

Case 2: 

In [ ]:
# mean function to move from stationarity to non-stationity in X and Y

def mean_stationary(s):
    c_mu = np.random.normal(8, 1)
    return np.tanh(c_mu * s - c_mu / 2)

In [ ]:
# historically dependent data
def hist_data(X, upper_limit, a, pred_points, s_0):
    """
    Function to generate historically dependent data

    Inputs:
    X: (n_vars, n_samples * n_tests, n_obs) array of samples
    upper_limit: upper limit for predictions
    a: strength of dependence
    pred_points: prediction points
    linear: extend of linear dependence

    Returns:
    Y: (n_samples * n_tests, n_obs) response variable that is historically dependent on X
    """
    if len(X.shape)==2:
        X_arr = X.reshape(1, X.shape[0], X.shape[1])
    else:
        X_arr = X
    Y = np.zeros((X_arr.shape[1], len(pred_points[pred_points <= upper_limit])))
    s, t = np.meshgrid(pred_points[pred_points <= upper_limit], pred_points[pred_points <= upper_limit])
    for i in range(X_arr.shape[1]):  # looping over samples
        sum_y = np.zeros(len(pred_points[pred_points <= upper_limit]))
        for p in range(X_arr.shape[0]):  # looping over parent variables
            beta_p = beta(s, t, linear=0)
            y = np.zeros(len(pred_points[pred_points <= upper_limit]))
            for i_t, t in enumerate(pred_points[pred_points <= upper_limit]):  # looping over time points of y
                #y[i_t] = trapezoid(X_arr[p, i][:i_t+1] * beta_p[:i_t+1, i_t])
                if i_t >= s_0:
                    y[i_t] = np.sum(X_arr[p, i][(i_t-s_0):i_t] * beta_p[(i_t-s_0):i_t, i_t]) / i_t
                else:
                    y[i_t] = 0

            sum_y += y
            Y[i] = sum_y

    return Y

Hyperparameters for data generation:

In [ ]:
n_trials = 200

n_samples = [100, 200, 300]
n_obs = 100
n_preds = 100
upper_limit = 1
period = 0.1
n_basis = 3
sd = 1
s_0 = 2

a_list = [0, 2, 4, 6, 8, 10]

pred_points = np.linspace(0, upper_limit, n_preds)
alpha = 0.05

n_intervals = 12
analyse = False
n_neighbours = 5
n_perms = 1000
make_K = K_ID
regressor = 'hist'

## Case 1:
Moving from linearity to non-linearity in the relationship between X and Y

Analysis over 200 independent trials:

In [ ]:
lens_DAGs_01_G = {}
lens_DAGs_10_G = {}
lens_DAGs_01_R = {}
lens_DAGs_10_R = {}

for n_sample in n_samples:
    print('n:', n_sample)

    lens_DAGs_01_G[n_sample] = {}
    lens_DAGs_10_G[n_sample] = {}
    lens_DAGs_01_R[n_sample] = {}
    lens_DAGs_10_R[n_sample] = {}

    for a in a_list:
        print('a:', a)

        lens_DAGs_01_G[n_sample][a] = []
        lens_DAGs_10_G[n_sample][a] = []
        lens_DAGs_01_R[n_sample][a] = []
        lens_DAGs_10_R[n_sample][a] = []

        for t in tqdm(range(n_trials)):

            # data generation
            obs_points_X = sample_points(n_sample, n_obs, upper_limit=upper_limit)
            X_mat = skfda_basis(n_sample, upper_limit, period, n_basis, sd).evaluate(obs_points_X, aligned=False).squeeze()
            X = spline_multi_sample(X_mat, obs_points_X, pred_points).evaluate(pred_points).squeeze() + np.random.normal(0, sd, size=(n_sample, n_preds))
            Y = hist_data_linear(X, upper_limit, a, pred_points, s_0) + np.random.normal(0, sd, size=(n_sample, n_preds))
            X_arr = np.asarray([X, Y])

            DAGs = {}
            DAGs_01_G = {}
            DAGs_10_G = {}
            p_values_G = {}
            p_values_01_G = {}
            p_values_10_G = {}

            # test Granger
            for i in range(X_arr.shape[1]):
                DAG, _, p_value, _ = granger(X_arr[:, i, :], alpha)
                DAGs[i] = DAG
                p_values_G[i] = p_value

                if DAG == {0: [], 1: 0}:
                    DAGs_01_G[i] = DAG
                    p_values_01_G[i] = p_value

                if DAG == {0: 1, 1: []}:
                    DAGs_10_G[i] = DAG
                    p_values_10_G[i] = p_value

            #print('Trial:', t)
            #print('X causes Y:', len(DAGs_01)/len(DAGs))
            #print('Y causes X:', len(DAGs_10)/len(DAGs))

            lens_DAGs_01_G[n_sample][a].append(len(DAGs_01_G)/len(DAGs))
            lens_DAGs_10_G[n_sample][a].append(len(DAGs_10_G)/len(DAGs))
            
            # test regression
            DAG_R, p_value_R = eval_candidate_DAGs(X_arr, pred_points, n_intervals, n_neighbours, n_perms, alpha, make_K, analyse, regressor, pd_graph=None)
            if DAG_R == {0: [], 1: [0]}:
                lens_DAGs_01_R[n_sample][a].append(DAG_R)
            elif DAG_R == {0: [1], 1: []}:
                lens_DAGs_10_R[n_sample][a].append(DAG_R)
            else:
                continue
            
        print('Granger (X -> Y) mean for a =', a, ':', np.mean(lens_DAGs_01_G[n_sample][a]))
        print('Granger (Y -> X) mean for a =', a, ':', np.mean(lens_DAGs_10_G[n_sample][a]))
        
        print('Regression (X -> Y) rate for a =', a, ':', len(lens_DAGs_01_R[n_sample][a]) / n_trials)
        print('Regression (Y -> X) rate for a =', a, ':', len(lens_DAGs_10_R[n_sample][a]) / n_trials)

In [ ]:
# save
results_01_G = open('results/causal/granger_linear_01.pkl', 'wb')
pickle.dump(lens_DAGs_01_G, results_01_G)
results_01_G.close()

results_10_G = open('results/causal/granger_linear_10.pkl', 'wb')
pickle.dump(lens_DAGs_10_G, results_10_G)
results_10_G.close()

results_01_R = open('results/causal/regression_linear_01_G.pkl', 'wb')
pickle.dump(lens_DAGs_01_R, results_01_R)
results_01_R.close()

results_10_R = open('results/causal/regression_linear_10_G.pkl', 'wb')
pickle.dump(lens_DAGs_10_R, results_10_R)
results_10_R.close()

## Case 2:
Moving from stationary to non-stationary time-series samples in X and Y

In [ ]:
lens_DAGs_01_C = {}
lens_DAGs_10_C = {}
lens_DAGs_01_R = {}
lens_DAGs_10_R = {}

for n_sample in n_samples:
    print('n:', n_sample)

    lens_DAGs_01_C[n_sample] = {}
    lens_DAGs_10_C[n_sample] = {}
    lens_DAGs_01_R[n_sample] = {}
    lens_DAGs_10_R[n_sample] = {}

    for a in a_list:
        print('a:', a)

        lens_DAGs_01_C[n_sample][a] = []
        lens_DAGs_10_C[n_sample][a] = []
        lens_DAGs_01_R[n_sample][a] = []
        lens_DAGs_10_R[n_sample][a] = []

        for t in tqdm(range(n_trials)):

            # data generation
            obs_points_X = sample_points(n_sample, n_obs, upper_limit=upper_limit)
            X_mat = a * mean_stationary(obs_points_X) + skfda_basis(n_sample, upper_limit, period, n_basis, sd).evaluate(obs_points_X, aligned=False).squeeze()
            X = spline_multi_sample(X_mat, obs_points_X, pred_points).evaluate(pred_points).squeeze() + np.random.normal(0, sd, size=(n_sample, n_preds))
            Y = a * mean_stationary(pred_points) + hist_data(X, upper_limit, 1, pred_points, s_0) + np.random.normal(0, sd, size=(n_sample, n_preds))
            X_arr = np.asarray([X, Y])

            DAGs = {}
            DAGs_01_C = {}
            DAGs_10_C = {}
            p_values_C = {}
            p_values_01_C = {}
            p_values_10_C = {}

            # test Granger
            for i in range(X_arr.shape[1]):
                DAG, _, p_value, _ = ccm_bivariate(X_arr[:, i, :], alpha)
                DAGs[i] = DAG
                p_values_C[i] = p_value

                if DAG == {0: [], 1: 0}:
                    DAGs_01_C[i] = DAG
                    p_values_01_C[i] = p_value

                if DAG == {0: 1, 1: []}:
                    DAGs_10_C[i] = DAG
                    p_values_10_C[i] = p_value

            #print('Trial:', t)
            #print('X causes Y:', len(DAGs_01)/len(DAGs))
            #print('Y causes X:', len(DAGs_10)/len(DAGs))

            lens_DAGs_01_C[n_sample][a].append(len(DAGs_01_C)/len(DAGs))
            lens_DAGs_10_C[n_sample][a].append(len(DAGs_10_C)/len(DAGs))
            
            # test regression
            DAG_R, p_value_R = eval_candidate_DAGs(X_arr, pred_points, n_intervals, n_neighbours, n_perms, alpha, make_K, analyse, regressor, pd_graph=None)
            if DAG_R == {0: [], 1: [0]}:
                lens_DAGs_01_R[n_sample][a].append(DAG_R)
            elif DAG_R == {0: [1], 1: []}:
                lens_DAGs_10_R[n_sample][a].append(DAG_R)
            else:
                continue
            
        print('CCM (X -> Y) mean for a =', a, ':', np.mean(lens_DAGs_01_C[n_sample][a]))
        print('CCM (Y -> X) mean for a =', a, ':', np.mean(lens_DAGs_10_C[n_sample][a]))
        
        print('Regression (X -> Y) rate for a =', a, ':', len(lens_DAGs_01_R[n_sample][a]) / n_trials)
        print('Regression (Y -> X) rate for a =', a, ':', len(lens_DAGs_10_R[n_sample][a]) / n_trials)

In [ ]:
# save
results_01_C = open('results/causal/ccm_linear_01_300.pkl', 'wb')
pickle.dump(lens_DAGs_01_C, results_01_C)
results_01_C.close()

results_10_C = open('results/causal/ccm_linear_10_300.pkl', 'wb')
pickle.dump(lens_DAGs_10_C, results_10_C)
results_10_C.close()

results_01_R = open('results/causal/regression_linear_01_C_300.pkl', 'wb')
pickle.dump(lens_DAGs_01_R, results_01_R)
results_01_R.close()

results_10_R = open('results/causal/regression_linear_10_C_300.pkl', 'wb')
pickle.dump(lens_DAGs_10_R, results_10_R)
results_10_R.close()